In [ ]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`
import $ivy.`org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0`

import $ivy.`org.apache.kafka:kafka-clients:3.5.1`
import $ivy.`org.testcontainers:testcontainers:1.19.3`
import $ivy.`org.testcontainers:kafka:1.19.3`

In [ ]:

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.kafka.clients.producer._

import org.testcontainers.containers.KafkaContainer
import org.testcontainers.utility.DockerImageName
import java.util.Properties

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._

val kafkaContainer = new KafkaContainer()

kafkaContainer.start()

val bootstrapServers = kafkaContainer.getBootstrapServers
val kafkaProps = new Properties()
kafkaProps.put("bootstrap.servers", bootstrapServers)
kafkaProps.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
kafkaProps.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")

val kafkaProducer = new KafkaProducer[String, String](kafkaProps)

def sendToKafka(topic: String, values: Seq[String]): Unit = {
 
  values.foreach { value =>
    val record = new ProducerRecord[String, String](topic, value)
    kafkaProducer.send(record)
  }
  kafkaProducer.flush()
}

def readFromKafkaSpark(topic: String): DataFrame = {
  spark.read
  .format("kafka")
  .option("kafka.bootstrap.servers", bootstrapServers)
  .option("subscribe", topic)
  .load()
  .selectExpr("CAST(value AS STRING) as message")
}

val kafkaTopic = "my-topic-scala"


In [ ]:
val data = Seq("hi", "there", "from", "scala")

sendToKafka(kafkaTopic, data)

In [ ]:
readFromKafkaSpark(kafkaTopic).show()